# Practica especial: Calculo de integrales utiles en molecular
## Por Facundo L Sanchez

In [ ]:
import numpy as np
from math import factorial
import matplotlib.pyplot as plt
from scipy.integrate import tplquad
from __future__ import division
from scipy.interpolate import spline
from scipy.interpolate import splrep,splev
import time as tm
%matplotlib inline

# La idea general de la practica es usar el paper de Roothaan, 1951 (http://dx.doi.org/10.1063/1.1748100) para recrear el valor de algunas integrales de un electron (overlap y atraccion nuclear) y de dos electrones (coulomb) en 2 centros usando STO. Luego voy a hacer la evaluacion numerica de las mismas usando coordenadas esferoidales prolatas y quadraturas en 3D (modulo scipy.integrate.tplquad) y comparar ambos resultados.

# Voy proponer una molecula de dos atomos (elegí que el atomo 1 sea Li ($Z_{Li} = 3$) y el atomo 2 sea H ($Z_{H} = 1$) )

# Los orbitales de Slater normalizados quedan definidos como:
# $\chi_{n,l,m}^{(a/b)}(\zeta,r_{a/b},\theta,\phi) = (2\zeta)^{n+\frac{1}{2}} [(2n)! ]^{-\frac{1}{2}}r_{a/b}^{n-1} e^{-\zeta r_{a/b}} S_{l,m}(\theta,\phi)$
## Con $r_{a/b} = |\vec{r} - \vec{R_{a/b}}|$ la distancia del electro medido desde el nucleo a o b, $\zeta = \frac{Z}{n}$ la carga nuclear efectiva y $S_{l,m}(\theta,\phi)$ los armonicos esfericos reales (normalizados)

## Como voy a trabajar con orbitales ns, el armonico esferico es $S_{0,0} = \frac{1}{\sqrt{4\pi}}$

# Defino los Slaters en distintos centros

In [ ]:
def Slater_a(n,l,m,Z,R,ksi,eta,phi):
    return (2*Z/n)**(n+0.5)*factorial(2*n)**(-0.5)*(0.5*(ksi+eta)*R)**(n-1)*np.exp(-Z/n*0.5*(ksi+eta)*R)/(np.pi*4)**0.5

def Slater_b(n,l,m,Z,R,ksi,eta,phi):
    return (2*Z/n)**(n+0.5)*factorial(2*n)**(-0.5)*(0.5*(ksi-eta)*R)**(n-1)*np.exp(-Z/n*0.5*(ksi-eta)*R)/(np.pi*4)**0.5


# Las integrales a resolver son:
# Un electron:
$
\begin{cases}
(\chi _a | \chi_b) = \int \chi_a \chi_b d\vec{r} \quad\quad\text{(overlap)} \\
(\chi _a|\frac{Z}{r_a} |\chi_b) = \int \chi_a \frac{Z}{r_a}\chi_b d\vec{r} \quad\quad\text{(atraccion nuclear)} \\
(\chi _b|\frac{Z}{r_a} |\chi'_b) = \int \chi_b \frac{Z}{r_a}\chi'_b d\vec{r} \quad\quad\text{(atraccion nuclear)} \\
\end{cases}
$
# Dos electrones:
$
\begin{align}
(\chi _a \chi _b | \frac{1}{r} |\chi'_a \chi'_b) = \int \chi _a(1) \chi _b(2) \frac{1}{r_{12}} \chi'_a(1) \chi'_b(2) d\vec{r_1}d\vec{r_2} \quad\quad\text{(coulomb)}
\end{align}
$

# Las coordenadas esferoidales:
$
\begin{cases} r_{a/b} = \frac{\xi \pm \eta}{2}R \\
\cos(\theta_{a/b}) = \frac{1 \pm \xi \eta}{\xi \pm \eta} \\
\phi = \phi_a = \phi_b
\end{cases}
$
## siendo R la distancia internuclear.
## El dominio de las nuevas variables
$
\begin{cases}
\xi \in [1,\infty )\\
\eta \in [-1,1]\\
\phi \in [0,2\pi]
\end{cases}
$
## y el elemento de volumen
$d\vec{r} = (\frac{R}{2})^3 (\xi^2-\eta^2)d\xi d\eta d\phi$.

# Defino entonces las funciones a integrar

In [ ]:
# Integral de overlap
integral_overlap = lambda ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2: Slater_a(n1,l1,m1,Z1,R,ksi,eta,phi)*Slater_b(n2,l2,m2,Z2,R,ksi,eta,phi)*(R/2)**3*(ksi**2-eta**2)

# Integral 1/ra con orbitales en 2 centros
integral_rminus = lambda ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2: Slater_a(n1,l1,m1,Z1,R,ksi,eta,phi)*Slater_b(n2,l2,m2,Z2,R,ksi,eta,phi)*(R/2)**3*(ksi**2-eta**2)/float(0.5*(ksi+eta)*R)

# Integral 1/ra con orbitales en 1 centro
integral_rminus_one_centre = lambda ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2: Slater_b(n2,l2,m2,Z2,R,ksi,eta,phi)*Slater_b(n2,l2,m2,Z2,R,ksi,eta,phi)*(R/2)**3*(ksi**2-eta**2)/(0.5*(ksi+eta)*R)


# Una vez definido todo, voy a arrancar los calculos. Siempre voy a usar distancia interatomica R = 3

# Voy a calcular $ (1s_a|1s_b)$

In [ ]:
# Defino los parametros
n1 = 1
l1 = 0
m1 = 0
n2 = 1
l2 = 0
m2 = 0
Z1 = 3
Z2 = 1
R = 3

In [ ]:
# Hago la integral triple
overlap = tplquad(lambda ksi,eta,phi: integral_overlap(ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2),0,2*np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)

print overlap[0]

# Comparo con la formula explicita en el paper ( pag 1449, eq (25) ). Como procedimiento general, para evaluar las formulas explicitas me tengo que definir los parametros de las eqs (15) y (16).

In [ ]:
# Defino los parametros usando los datos de mis atomos
ksia = Z1/n1
ksib = Z2/n2
ksi = 1/2*(ksia+ksib)
tau = (ksia-ksib)/(ksia+ksib)
rho = ksi*R
k = (ksia**2+ksib**2)/(ksia**2-ksib**2)
rhoa = ksia*R
rhob = ksib*R

In [ ]:
sa1sb1 = (1-tau**2)**0.5/(tau*rho)*(-(1-k)*(2*(1+k)+rhoa)*np.exp(-rhoa)+(1+k)*(2*(1-k)+rhob)*np.exp(-rhob))
print sa1sb1

In [ ]:
# Calculo la diferencia entre ambos resultados
np.abs(overlap[0]-sa1sb1)

# Voy a calcular $ (1s_a|2s_b)$

In [ ]:
n1 = 1
l1 = 0
m1 = 0
n2 = 2
l2 = 0
m2 = 0
Z1 = 3
Z2 = 1
R = 3

In [ ]:
overlap = tplquad(lambda ksi,eta,phi: integral_overlap(ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2),0,2*np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)

print overlap[0]


# Comparo con la formula analitica

In [ ]:
ksia = Z1/n1
ksib = Z2/n2
ksi = 1/2*(ksia+ksib)
tau = (ksia-ksib)/(ksia+ksib)
rho = ksi*R
k = (ksia**2+ksib**2)/(ksia**2-ksib**2)
rhoa = ksia*R
rhob = ksib*R


In [ ]:
sa1sb2=(1-tau**2)**(1/2)/(3**0.5*tau*rho)*(-(1-k)*(2*(1+k)*(2-3*k)+(1-2*k)*rhoa)*np.exp(-rhoa)+\
                                         (1+k)*(2*(1-k)*(2-3*k)+4*(1-k)*rhob+rhob**2)*np.exp(-rhob))
print sa1sb2

In [ ]:
# Calculo la diferencia entre ambos resultados
np.abs(overlap[0]-sa1sb2)

# Ahora voy a calcular las atracciones nucleares

# Voy a calcular $(2s_a|\frac{1}{r_a}|1s_b)$

In [ ]:
n1 = 2
l1 = 0
m1 = 0
n2 = 1
l2 = 0
m2 = 0
Z1 = 3
Z2 = 1
R = 3

In [ ]:
rminus = tplquad(lambda ksi,eta,phi: integral_rminus(ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2),-np.pi,np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)

print rminus[0]


# Usando la formula analitica

In [ ]:
ksia = Z1/n1
ksib = Z2/n2
ksi = 1/2*(ksia+ksib)
tau = (ksia-ksib)/(ksia+ksib)
rho = ksi*R
k = (ksia**2+ksib**2)/(ksia**2-ksib**2)
rhoa = ksia*R
rhob = ksib*R

In [ ]:
sa1sb1 = (1-tau**2)**0.5/(tau*rho)*(-(1-k)*(2*(1+k)+rhoa)*np.exp(-rhoa)+(1+k)*(2*(1-k)+rhob)*np.exp(-rhob))

print (1/3**0.5)*ksi*(1+tau)*sa1sb1

In [ ]:
# Calculo la diferencia entre ambos resultados
np.abs(rminus[0]-(1/3**0.5)*ksi*(1+tau)*sa1sb1)

# Voy a calcular $(2s_a|\frac{1}{r_a}|2s_b)$

In [ ]:
n1 = 2
l1 = 0
m1 = 0
n2 = 2
l2 = 0
m2 = 0
Z1 = 3
Z2 = 1
R = 3

In [ ]:
rminus = tplquad(lambda ksi,eta,phi: integral_rminus(ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2),0,2*np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)

print rminus[0]


# Usando la formula explicita

In [ ]:
ksia = Z1/n1
ksib = Z2/n2
ksi = 1/2*(ksia+ksib)
tau = (ksia-ksib)/(ksia+ksib)
rho = ksi*R
k = (ksia**2+ksib**2)/(ksia**2-ksib**2)
rhoa = ksia*R
rhob = ksib*R

In [ ]:
sa1sb2=(1-tau**2)**(0.5)/(3**0.5*tau*rho)*(-(1-k)*(2*(1+k)*(2-3*k)+(1-2*k)*rhoa)*np.exp(-rhoa)+\
                                         (1+k)*(2*(1-k)*(2-3*k)+4*(1-k)*rhob+rhob**2)*np.exp(-rhob))
print (1/3**0.5)*ksi*(1+tau)*sa1sb2

In [ ]:
# Calculo la diferencia entre ambos resultados
np.abs(rminus[0]-(1/3**0.5)*ksi*(1+tau)*sa1sb2)

# Ahora quiero calcular la integral de Coulomb $(1s_a1s_b|\frac{1}{r}|1s_a1s_b)$

## Voy a seguir el procedimiento que hace en el paper, en el cual primero calcula el potencial de interaccion de la distribucion de carga de un electron con el nucleo y luego, con este potencial, se integra para el otro electron, de manera que queden integrales de un electron cada vez. Basicamente:
## $(1s_a1s_b|\frac{1}{r}|1s_a1s_b) = [\Omega_a|\Omega_b] = \int \int \frac{\Omega_a(1)\Omega_b(2)}{r_{12}} d\vec{r_1}d\vec{r_2} = \int \Omega_b(2) U(2) d\vec{r_2}$ ,
## donde $\Omega$ es la distribucion de carga de un electron con respecto a un centro. Entonces, lo que hace es integrar alguna de los 2 juegos de variables para obtener un potencial U evaluado en cada punto y luego integrar este potencial con respecto al otro juego de variables.

# Primero tenemos que hallar el potencial $U_{1s}$, pero este se puede escribir de una manera muy simple en funcion de la distribucion de carga $[a|1S_b]$ (eq (32)), entonces vamos con las integrales de distribucion de carga. En nuestro caso, por los orbitales elegidos tenemos $[a|1s_b1's_b] = [a|1S_b]$ (eq (12))

# Defino los parametros

In [ ]:
Z1 = 3
Z2 = 1
n1 = 1
l1 = 0
m1 = 0
n2 = 1
l2 = 0
m2 = 0
R = 3

# Evaluo la integral $[a|1S_b]$ en un punto, para corroborar con la forma analitica

In [ ]:
nuclear_att = tplquad(lambda ksi,eta,phi: integral_rminus_one_centre(ksi,eta,phi,R,n1,l1,m1,Z1,n2,l2,m2,Z2),0,2*np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)
print nuclear_att[0]

# Comparo con la formula analitica ( eq (31) )

In [ ]:
ksihat = 1/2*(2*Z2/n2)
rho = R*ksihat

In [ ]:
a_1Sb = ksihat/rho*(1-(1+rho)*np.exp(-2*rho))
print a_1Sb

In [ ]:
# Calculo la diferencia
np.abs(nuclear_att[0] - a_1Sb)

# Ahora, a partir de la integral $[a|1S_b]$ evaluando en cada distancia interatomica, me construyo el potencial correspondiente $U_{1S}(r,\theta,\phi)$

In [ ]:
U1s_func = lambda r: tplquad(lambda ksi,eta,phi: integral_rminus_one_centre(ksi,eta,phi,r,n1,l1,m1,Z1,n2,l2,m2,Z2),0,2*np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)[0]

# Esta integral es bastante costosa de hacer punto a punto, por lo que propongo construirme una aproximacion de la misma haciendo un spline sobre un grid logaritmico

# Voy a la forma aproximada del potencial para ver como hago mi grid

In [ ]:
r = np.logspace(np.log10(1e-3),np.log10(10000),20)
U_test = np.zeros(20)
for i in range(20):
    U_test[i] = U1s_func(r[i])

In [ ]:
plt.plot(r,U_test,'r.-')
plt.yscale('log')

# Pareceria ser que para el ultimo punto r = 10000, la integracion no estaria siendo tan buena, o decae abruptamente a cero. De cualquier manera, voy a poner como limite de mi intervalo de sampleo r = 4000, que es el anteultimo punto del grafico

In [ ]:
plt.plot(r[:15],U_test[:15],'r.-')
plt.yscale('log')

# Ademas, a partir de r = 100, parece tener un caracter lineal. Entonces voy a samplear en 2 intervalos, uno de $10^{-3}$ a 100 con 100 puntos, y otro de 100 a 4000 con 20 puntos, la idea es poder reproducir mejor el comportamiento cerca del r = 0, ya que luego se comporta lineal, por eso tengo menos puntos en el otro rango

# Me construyo un grid sobre el cual samplear la integral

In [ ]:
r = np.logspace(np.log10(1e-3),np.log10(100),100)
r_prime = np.logspace(np.log10(101),np.log10(4000),20)
r = np.append(r,r_prime)
print r

# Voy a evaluar la integral en el grid para obtener el potencial (guarda que tarda unos 5 minutos aprox.)

In [ ]:
time_st = tm.time()
U1s = np.zeros(len(r))
for i in range(len(r)):
    if i%10 == 0:
        print i
    U1s[i] = U1s_func(r[i])
time_end = tm.time()
print "Tiempo de ejecucion:",int(time_end - time_st),"seg"

# Voy a graficar el potencial sampleado y el analitico (eq (33)) en log-log para comparar

In [ ]:
plt.plot(r,U1s,'r.',label='Calculado')
plt.plot(r,1/r*(1-(1+r*ksihat)*np.exp(-2*r*ksihat)),label='Analitico')
plt.legend(loc='best')
plt.yscale('log')
plt.xscale('log')
plt.title("Comparacion Potencial Calculado y Analitico")

# Grafico la diferencia entre ambos

In [ ]:
plt.plot(np.abs(U1s-1/r*(1-(1+r*ksihat)*np.exp(-2*r*ksihat))))
plt.yscale('log')
plt.title("Diferencia entre potencial Calculado y analitico")

# Hago el spline cubico sobre el potencial sampleado y grafico evaluando en otros valores

In [ ]:
#U_func = [U1s,r]
# Hago el spline
U = splrep(r,U1s)
interpol = splev(r1,U)
# Nueva grilla
r1 = np.linspace(1e-4,200,1000)
plt.plot(r1,1/r1*(1-(1+r1*ksihat)*np.exp(-2*r1*ksihat)),label='Analitico')
plt.plot(r1,interpol,'r--', label='Spline')
plt.legend(loc='best')
plt.xscale('log')
plt.yscale('log')
plt.title("Comparacion Spline y analitico")

# Grafico la diferencia entre el spline y el analitico

In [ ]:
plt.plot(np.abs(interpol-1/r1*(1-(1+r1*ksihat)*np.exp(-2*r1*ksihat))))
plt.yscale('log')
plt.title("Diferencia entre Potencial via Spline y analitico")

# Ya tengo todo para hallar $[1S_a|1S_b]$ que, por la eq (30) se que $ [1S_a|1S_b] = (1s_a1s_b|\frac{1}{r}|1s_a1s_b) $, que es la integral de Coulomb que buscaba

In [ ]:
Z1 = 3
Z2 = 1
n1 = 1
l1 = 0
m1 = 0
n2 = 1
l2 = 0
m2 = 0
R = 3

# Hago la integral numerica usando el spline

In [ ]:
def integral_coulomb(U_func,R,n1,l1,m1,n2,l2,m2,Z1,Z2,ksi,eta,phi):
    return Slater_a(n1,l1,m1,Z1,R,ksi,eta,phi)*Slater_a(n1,l1,m1,Z1,R,ksi,eta,phi)*(R/2)**3*(ksi**2-eta**2)*float(splev(0.5*R*(ksi-eta),U_func))

In [ ]:
coulomb_res_calc = tplquad(lambda ksi,eta,phi: integral_coulomb(U,R,n1,l1,m1,n2,l2,m2,Z1,Z2,ksi,eta,phi),0,2*np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)
print coulomb_res_calc

# Uso la expresion analitica del potencial (eq (33)) y calculo la integral

In [ ]:
integral_coulomb = lambda U_func,R,n1,l1,m1,n2,l2,m2,Z1,Z2,ksi,eta,phi: Slater_a(n1,l1,m1,Z1,R,ksi,eta,phi)*Slater_a(n1,l1,m1,Z1,R,ksi,eta,phi)*1/(0.5*R*(ksi-eta))*(1-(1+(0.5*R*(ksi-eta))*ksihat)*np.exp(-2*(0.5*R*(ksi-eta))*ksihat))*(R/2)**3*(ksi**2-eta**2)
coulomb_res = tplquad(lambda ksi,eta,phi: integral_coulomb(U1s_func,R,n1,l1,m1,n2,l2,m2,Z1,Z2,ksi,eta,phi),0,2*np.pi, lambda phi: -1, lambda phi: 1, lambda phi,eta: 1, lambda phi,eta: np.inf)
print coulomb_res

# Chequeo con la formula del paper para $[1S_a|1S_b]$ (pag 1453)

In [ ]:
ksia = Z1/n1
ksib = Z2/n2
ksi = 1/2*(ksia+ksib)
tau = (ksia-ksib)/(ksia+ksib)
rho = ksi*R
k = (ksia**2+ksib**2)/(ksia**2-ksib**2)
rhoa = ksia*R
rhob = ksib*R

In [ ]:
Sa1Sb1 = ksi/rho*(1-(1-k)**2*(1/4*(2+k)+1/4*rhoa)*np.exp(-2*rhoa)-(1+k)**2*(1/4*(2-k)+1/4*rhob)*np.exp(-2*rhob))
print Sa1Sb1

# Comparaciones entre los valores obtenidos con la integracion via spline y la integracion via formula analitica

In [ ]:
print "Diferencia entre resultado analitico e integracion via spline: ",np.abs(coulomb_res_calc[0]-Sa1Sb1)
print "Diferencia entre resultado analitico e integracion con el potencial analitico: ",np.abs(coulomb_res[0]-Sa1Sb1)

# Si bien se puede ver que la diferencia entre el calculo integrando el spline  vs integrando el potencial analitico es notable, pues hay 7 ordendes de magnitud de diferencia, podemos ver que usando el spline pude obtener 6 cifras, lo cual me parece un buen resultado, considerando el tiempo que tardó en calcular la integral con el spline y el tiempo que tardaría en calcular el potencial integrando punto a punto directamente en la integral, o sea, haciendo la integral sextuple (lo tuve corriendo aprox 6 hs y no terminó)